<a href="https://colab.research.google.com/github/Kiligmzq6/GPT/blob/main/GLM_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from zhipuai import ZhipuAI
from google.colab import userdata

In [16]:
client = ZhipuAI(api_key=userdata.get('zhipukey'))


def get_relation(text):
  tools = [
      {
          "type": "function",
          "function": {
              "name": "NER",
              "description": "根据提供的文本，抽取出 主体、客体、关系、关系分类",
              "parameters": {
                  "type": "object",
                  "properties": {
                      "主体": {
                          "type": "string",
                          "description": "文本主体",
                      },
                      "客体": {
                          "type": "string",
                          "description": "文本客体",
                      },
                      "关系": {
                          "type": "string",
                          "description": "主客体关系",
                      },
                      "关系分类": {
                          "type": "string",
                          "description": "主客体关系分类",
                      },
                  },
                  "required": ["主体", "客体", "关系", "关系分类"],
              },
          }
      }
  ]
  messages = [
      {
          "role": "system",
          "content": "不要假设或猜测传入函数的参数值。如果文本的描述不明确，返回无即可"
      },
      {
          "role": "user",
          "content": text
      }
    ]

  response = client.chat.completions.create(
      model="glm-3-turbo",
      messages=messages,
      tools=tools,
      tool_choice="auto",
  )

  if not response.choices or not response.choices[0].message.tool_calls:
    return None

  return response.choices[0].message.tool_calls[0].function.arguments


In [17]:
text = "小明把书给了小红。"
arguments = get_relation(text)
print(arguments)

{"主体":"小明","关系":"给","关系分类":"转让","客体":"书"}


In [18]:
text = "太阳照在花朵上。"
arguments = get_relation(text)
print(arguments)

{"主体":"太阳","关系":"照在","关系分类":"阳光照射","客体":"花朵"}


In [30]:
def get_relation_one_shot(text):
  tools = [
      {
          "type": "function",
          "function": {
              "name": "NER",
              "description": "根据提供的文本，抽取出 主体、客体、关系、关系分类",
              "parameters": {
                  "type": "object",
                  "properties": {
                      "主体": {
                          "type": "string",
                          "description": "文本主体",
                      },
                      "客体": {
                          "type": "string",
                          "description": "文本客体",
                      },
                      "关系": {
                          "type": "string",
                          "description": "主客体关系",
                      },
                      "关系分类": {
                          "type": "string",
                          "description": "主客体关系分类",
                      },
                  },
                  "required": ["主体", "客体", "关系", "关系分类"],
              },
          }
      }
  ]
  messages = [
      {
          "role": "system",
          "content": "不要假设或猜测传入函数的参数值。如果文本的描述不明确，返回无即可"
      },
      {
          "role": "system",
          "content": "一个样例：输入“小明把书给了小红”，输出：{'主体': '小明', '客体': '小红', '关系': '把书给了', '关系分类': '赠予'}"
      },
      {
          "role": "user",
          "content": text
      }
    ]

  response = client.chat.completions.create(
      model="glm-3-turbo",
      messages=messages,
      tools=tools,
      tool_choice="auto",
  )

  if not response.choices or not response.choices[0].message.tool_calls:
    return None

  return response.choices[0].message.tool_calls[0].function.arguments


In [31]:
text = "小明把书给了小红。"
arguments = get_relation_one_shot(text)
print(arguments)

{"主体":"小明","关系":"给了","关系分类":"赠予","客体":"书"}


In [32]:
text = "小猫在院子里玩耍。"
arguments = get_relation_one_shot(text)
print(arguments)

{"主体":"小猫","关系":"在...玩耍","关系分类":"活动地点","客体":"院子"}
